## Save Dataset conditioned at time of maximum TREFHT at each coordinate

In [2]:
import scipy.optimize
import xarray as xr
import pandas as pd
import matplotlib
import numpy as np
import cartopy.mpl.ticker as cticker
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import metpy
import scipy.stats as sts
from glob import glob
import sys
import cftime
import datetime
sys.path.append('/home/users/jamd1/Isca/')
import warnings
import isca_tools
from isca_tools import cesm
from isca_tools.plot import fig_resize, update_fontsize, update_linewidth, savefig
from isca_tools.utils import area_weighting, print_ds_var_list
from isca_tools.utils.base import round_any
from isca_tools.utils.constants import L_v, c_p, g, R, kappa, lapse_dry
from isca_tools.convection.base import lcl_metpy, dry_profile_temp, moist_profile
from isca_tools.thesis.lapse_theory import get_bulk_lapse_rate, reconstruct_temp, interp_var_at_pressure
from isca_tools.utils.base import top_n_peaks_ind
from isca_tools.utils.moist_physics import sphum_sat
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from statsmodels.stats.multitest import fdrcorrection
from tqdm.notebook import tqdm
import scipy
from geocat.comp.interpolation import interp_hybrid_to_pressure
from isca_tools.cesm.load import jasmin_archive_dir, jasmin_surf_geopotential_file
import copy
import os

print(os.getcwd())
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/home/users/jamd1/Isca/jobs/cesm/3_hour/hottest_time


### Play with getting n hottest times
Must be separated by more than 15 hours, so 1 value per day. Cannot use groupby because time varies with location.

This is playing around before used in hottest_quant/save_info

In [5]:
exp_name = ['e.e20.E1850TEST.f09_g17.3hour_output', 'e.e20.E1850TEST.f09_g17.co2_2x_3hour_output']
n_exp = len(exp_name)
var_keep = ['CAPE', 'PRECC', 'PRECL', 'PS', 'Q', 'QREFHT', 'T', 'TREFHT', 'TS', 'Z3']

In [61]:
i = 0
ds = cesm.load_dataset(exp_name[i], hist_file=1, year_files=61, month_files=1)[var_keep]

In [62]:
ds = ds.load()

In [63]:
hour_spacing = 13
time_dt = (ds.time[1]-ds.time[0]) / np.timedelta64(1, 'h')
ind_spacing = int(np.ceil(hour_spacing/time_dt))

n_sample = 5
peak_values = xr.apply_ufunc(
    top_n_peaks_ind,
    ds.TREFHT,
    input_core_dims=[["time"]],
    output_core_dims=[["sample"]],
    vectorize=True,
    kwargs={"n": n_sample, "min_ind_spacing": ind_spacing},
    dask="parallelized",
    output_dtypes=[int],
)
peak_values = peak_values.assign_coords(sample=np.arange(1, n_sample+1))

### Script to save data

In [ ]:
exp_name = ['e.e20.E1850TEST.f09_g17.3hour_output', 'e.e20.E1850TEST.f09_g17.co2_2x_3hour_output']
n_exp = len(exp_name)
var_keep = ['CAPE', 'PRECC', 'PRECL', 'PS', 'Q', 'QREFHT', 'T', 'TREFHT', 'TS', 'Z3']

for i in range(n_exp):
    ds = cesm.load_dataset(exp_name[i], hist_file=1)
    print('Lazy loaded dataset')
    gw = ds.gw.isel(time=0).load()
    hyam = ds.hyam.isel(time=0).load()
    hybm = ds.hybm.isel(time=0).load()
    p0 = float(ds.P0.isel(time=0))
    ds = ds[var_keep].load()
    print('Fully loaded dataset')
    t500_zonal_daily_av = interp_var_at_pressure(ds.T.resample(time="1D").mean(dim="time").mean(dim='lon'), np.atleast_1d(500*100),
                                                 ds.PS.resample(time="1D").mean(dim="time").mean(dim='lon'), hyam, hybm, p0).T
    print('Computed daily zonal average T500')
    t500_zonal_daily_av = t500_zonal_daily_av.load()
    print('Loaded daily zonal average T500')
    idx_max = ds['TREFHT'].argmax(dim='time')
    ds_out = ds.isel(time=idx_max)
    ds_out['time_max'] = ds.time.isel(time=idx_max)
    ds_out['T500_zonal_daily_av'] = t500_zonal_daily_av
    ds_out['gw'] = gw
    ds_out['hyam'] = hyam
    ds_out['hybm'] = hybm
    ds_out['P0'] = p0
    print('Created ds_out')
    ds_out.to_netcdf(f'/gws/nopw/j04/global_ex/jamd1/cesm/CESM2.1.3/archive/{exp_name[i]}/processed/ds_hot.nc')
    print('Saved ds_out')

Lazy loaded dataset
Fully loaded dataset
Computed daily zonal average T500
Loaded daily zonal average T500
Created ds_out
Saved ds_out
Lazy loaded dataset
